In [10]:
import json
import pandas as pd
import os

import PySAM
import PySAM.Pvsamv1 as pv_model
import PySAM.Grid as grid
import PySAM.Utilityrate5 as utility_rate
import PySAM.ResourceTools as tools

from functools import partial
import itertools
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from operator import sub
import seaborn as sns

In [11]:
def get_cmod_json(json_file_path):
    with open(json_file_path, 'r') as f:
        dict = json.load(f)
    return dict

In [12]:
states = pd.read_excel("USDA GH Sq Ft by State 1.xlsx", header=1)

locs = pd.read_csv("us_county_latlng.csv", header=0)



In [13]:
print(states.columns)
print(locs.columns)

Index(['State', 'GH Operations', 'GH Footprint Area (sq ft)',
       'GH Roof Area (sq ft)', 'GH Roof Area - 50% coverage (sq ft)', 'Lat',
       'Long'],
      dtype='object')
Index(['fips_code', 'name', 'lng', 'lat'], dtype='object')


In [14]:
data_folder = "SAM_files/"
output_folder = "results/"

data_cols = ["state", "lat", "long", "sq_ft", "Energy Produced (kWh)", "Energy per area (kWh/m^2)", "Energy within county (kWh)"]



In [46]:
full_data = []
sam_api_key = "RkdVzEGa7mlEGqV9jzwjhrh95ACZOr6PFr0K0yTc"
sam_email = "brian.mirletz@nrel.gov"
error_list = []

for index, row in states.iterrows():
    pv = pv_model.default("FlatPlatePVNone")

    name = row['State']
    sq_ft = float(row["GH Roof Area - 50% coverage (sq ft)"])
    if sq_ft > 0:
        try:
            lat = row['Lat']
            long = row['Long']


            print(lat, long, name)



            nsrdbfetcher = tools.FetchResourceFiles(
                            tech='solar',
                            nrel_api_key=sam_api_key,
                            nrel_api_email=sam_email,
                            resource_dir="weather_data")

            lon_lats = []

            lon_lat = (long, lat)
            lon_lats.append(lon_lat)

            nsrdbfetcher.fetch(lon_lats)
        except Exception:
            print("Error with state: ", name)
            error_list.append(name)

print("Errors with ", error_list)

33.98087061 -86.56700640 1009.0

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_33.98087061_-86.56700640_psm3-tmy_60_tmy.csv
34.13183982 -86.86718877 1043.0

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_34.13183982_-86.86718877_psm3-tmy_60_tmy.csv
34.46023769 -85.80378574 1049.0

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_34.46023769_-85.80378574_psm3-tmy_60_tmy.csv
33.72117266 -87.73924331 1057.0

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_33.72117266_-87.73924331_psm3-tmy_60_tmy.csv
34.44238135 -87.84328300 1059.0

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_34.44238135_-87.84328300_psm3-tmy_60_tmy.csv
32.76259480 -87.62930576 1065.

In [17]:
full_data = []
sam_api_key = "RkdVzEGa7mlEGqV9jzwjhrh95ACZOr6PFr0K0yTc"
sam_email = "brian.mirletz@nrel.gov"

for index, row in states.iterrows():
    pv = pv_model.default("FlatPlatePVNone")

    pv_json = get_cmod_json(data_folder + "Biomatch_1_pvsamv1.json")

    name = row['State']
    sq_ft = float(row["GH Roof Area - 50% coverage (sq ft)"])
    if sq_ft > 0:
        lat = row['Lat']
        long = row['Long']


        print(lat, long, name)

        for k, v in pv_json.items():
            if "adjust_" in k:
                k = k.replace("adjust_", "")
            try:
                pv.value(k, v)
            except AttributeError:
                print("Error ", k)

        nsrdbfetcher = tools.FetchResourceFiles(
                        tech='solar',
                        nrel_api_key=sam_api_key,
                        nrel_api_email=sam_email,
                        resource_dir="weather_data")

        lon_lats = []

        lon_lat = (long, lat)
        lon_lats.append(lon_lat)

        nsrdbfetcher.fetch(lon_lats)

        pv.SolarResource.solar_resource_file = "weather_data/nsrdb_" + str(lat) + "_" + str(long) +"_psm3-tmy_60_tmy.csv"

        pv.execute()

        annual_energy = pv.Outputs.annual_energy
        kwh_per_m2 = annual_energy / 2.0
        kwh_per_state = kwh_per_m2 * sq_ft / 10.7639  # 10.76 sq ft per meter.

        values = [name, float(lat), float(long), sq_ft, annual_energy, kwh_per_m2, kwh_per_state]

        full_data.append(values)



final_df = pd.DataFrame(data=full_data, columns=data_cols)
final_df = final_df.set_index('state')

final_df.to_csv("mind_the_gap_energy_outputs_state.csv")


44.967243 -103.771556 United States 

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_44.967243_-103.771556_psm3-tmy_60_tmy.csv
32.31823 -86.902298 Alabama 

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_32.31823_-86.902298_psm3-tmy_60_tmy.csv
58.45091866 -134.1764399 Alaska 

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_58.45091866_-134.1764399_psm3-tmy_60_tmy.csv
34.048927 -111.093735 Arizona 

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_34.048927_-111.093735_psm3-tmy_60_tmy.csv
34.799999 -92.199997 Arkansas 

Starting data download for solar using 1 thread workers.
File already exists. Skipping download: weather_data\nsrdb_34.799999_-92.199997_psm3-tmy_60_tmy.csv
36.778259 -119.417931 California 

Starting 

KeyboardInterrupt: 

In [51]:
print(sum(final_df["Energy within county (kWh)"]))

296787831.84558105


In [32]:
final_df.index.isin(('100kW', any()))

TypeError: any() takes exactly one argument (0 given)